In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import codecs
!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
import json 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium 
print('Complete')

--2020-08-29 16:46:30--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56490 (55K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M’

100%[======================================>] 56,490      --.-K/s   in 0.05s   

2020-08-29 16:46:31 (1.09 MB/s) - ‘List_of_postal_codes_of_Canada:_M’ saved [56490/56490]

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge


# Toronto Data


In [6]:
toronto = codecs.open('List_of_postal_codes_of_Canada:_M', encoding='utf-8')
html = toronto.read()
soup = BeautifulSoup(html, 'html.parser')
toronto_df = (pd.read_html(str(soup.table))[0])
toronto_df.drop(axis=0, index=0, inplace=True)
toronto_df

,Postal Code,Borough,Neighbourhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
10,M2B,Not assigned,Not assigned


# Organize and Clean Data

In [7]:
toronto_columns = ['PostalCode', 'Borough', 'Neighbourhood'] 

#remove not assigned
toronto_df = toronto_df[toronto_df['Borough'] != "Not assigned"].reset_index(drop=True) 
toronto_df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Long and Lat


In [8]:
!wget http://cocl.us/Geospatial_data

--2020-08-29 17:03:49--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.55.161.7
Connecting to cocl.us (cocl.us)|169.55.161.7|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-08-29 17:03:49--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.55.161.7|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-29 17:03:50--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-29 17:03:50--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mv

In [9]:
toronto_geo = pd.read_csv('http://cocl.us/Geospatial_data')
toronto_geo.head()
toronto_geo.columns = ['Postal Code', 'Latitude', 'Longitude']
toronto_geo

#merge data
toronto_merged_df = pd.merge(toronto_df, toronto_geo, on= 'Postal Code')
toronto_merged_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Foursquare Info

In [10]:
CLIENT_ID = 'LPJNPAKLUKGOPLMLQK5VN4IIXDGZG4TD1G4CIRUZ2041M32T' # your Foursquare ID
CLIENT_SECRET = 'NDO54RFAMEAFLB3J2QRSB0B5K5DM3OI5DRXATAYQCWGG3XQN' # your Foursquare Secret
VERSION = '20200815' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LPJNPAKLUKGOPLMLQK5VN4IIXDGZG4TD1G4CIRUZ2041M32T
CLIENT_SECRET:NDO54RFAMEAFLB3J2QRSB0B5K5DM3OI5DRXATAYQCWGG3XQN


In [11]:
toronto_merged_df.loc[2, 'Neighbourhood']
neighbourhood_lat = toronto_merged_df.loc[2, 'Latitude'] # neighborhood latitude value
neighbourhood_long = toronto_merged_df.loc[2, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_merged_df.loc[2, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_lat, 
                                                               neighbourhood_long))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [12]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_lat, 
    neighbourhood_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LPJNPAKLUKGOPLMLQK5VN4IIXDGZG4TD1G4CIRUZ2041M32T&client_secret=NDO54RFAMEAFLB3J2QRSB0B5K5DM3OI5DRXATAYQCWGG3XQN&v=20200815&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f4a8ac1fa0ba44151dd26b0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 49,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

# Category Types

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
    
toronto_nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
toronto_nearby_venues = toronto_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
toronto_nearby_venues['venue.categories'] = toronto_nearby_venues.apply(get_category_type, axis=1)

# clean columns
toronto_nearby_venues.columns = [col.split(".")[-1] for col in toronto_nearby_venues.columns]

toronto_nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980
5,Corktown Common,Park,43.655618,-79.356211
6,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
7,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
8,The Distillery Historic District,Historic Site,43.650244,-79.359323
9,The Extension Room,Gym / Fitness Center,43.653313,-79.359725


In [16]:
print('{} venues were returned by Foursquare.'.format(toronto_nearby_venues.shape[0]))

49 venues were returned by Foursquare.


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = getNearbyVenues(names=toronto_merged_df['Neighbourhood'],
                                   latitudes=toronto_merged_df['Latitude'],
                                   longitudes=toronto_merged_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [19]:
toronto_venues

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

There are 269 uniques categories.


,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.055556
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [22]:
num_top_venues = 10

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                             venue  freq
0                           Lounge  0.25
1                   Breakfast Spot  0.25
2                     Skating Rink  0.25
3        Latin American Restaurant  0.25
4                Accessories Store  0.00
5        Middle Eastern Restaurant  0.00
6  Molecular Gastronomy Restaurant  0.00
7       Modern European Restaurant  0.00
8                Mobile Phone Shop  0.00
9               Miscellaneous Shop  0.00


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.22
1                 Gym  0.11
2      Sandwich Place  0.11
3                 Pub  0.11
4         Coffee Shop  0.11
5                Pool  0.11
6        Skating Rink  0.11
7            Pharmacy  0.11
8       Metro Station  0.00
9  Mexican Restaurant  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.09
1                Coffee Shop  0.09
2              Shopping Mal

                 venue  freq
0                  Gym  0.11
1          Coffee Shop  0.07
2           Beer Store  0.07
3           Restaurant  0.07
4  Japanese Restaurant  0.07
5     Asian Restaurant  0.04
6       Clothing Store  0.04
7          Supermarket  0.04
8       Baseball Field  0.04
9           Smoke Shop  0.04


----Dorset Park, Wexford Heights, Scarborough Town Centre----
                             venue  freq
0                Indian Restaurant   0.4
1                        Pet Store   0.2
2               Chinese Restaurant   0.2
3            Vietnamese Restaurant   0.2
4               Miscellaneous Shop   0.0
5              Moroccan Restaurant   0.0
6              Monument / Landmark   0.0
7  Molecular Gastronomy Restaurant   0.0
8       Modern European Restaurant   0.0
9                Mobile Phone Shop   0.0


----Downsview----
            venue  freq
0   Grocery Store  0.20
1            Park  0.13
2  Baseball Field  0.07
3         Airport  0.07
4            Bank  0.07
5 

                           venue  freq
0                           Café  0.07
1                    Coffee Shop  0.06
2             Mexican Restaurant  0.04
3          Vietnamese Restaurant  0.04
4  Vegetarian / Vegan Restaurant  0.04
5                            Bar  0.04
6                         Bakery  0.03
7                   Dessert Shop  0.03
8                    Pizza Place  0.03
9            Dumpling Restaurant  0.03


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                             venue  freq
0                             Park  0.33
1                   Sandwich Place  0.33
2                Mobile Phone Shop  0.33
3                Accessories Store  0.00
4        Middle Eastern Restaurant  0.00
5              Monument / Landmark  0.00
6  Molecular Gastronomy Restaurant  0.00
7       Modern European Restaurant  0.00
8               Miscellaneous Shop  0.00
9               Mexican Restaurant  0.00


----Lawrence Manor, Lawrence Heights

                             venue  freq
0                           Garden   1.0
1                Accessories Store   0.0
2        Middle Eastern Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0
5       Modern European Restaurant   0.0
6                Mobile Phone Shop   0.0
7               Miscellaneous Shop   0.0
8               Mexican Restaurant   0.0
9                 Malay Restaurant   0.0


----Rouge Hill, Port Union, Highland Creek----
                             venue  freq
0                              Bar   1.0
1                Accessories Store   0.0
2               Miscellaneous Shop   0.0
3              Moroccan Restaurant   0.0
4              Monument / Landmark   0.0
5  Molecular Gastronomy Restaurant   0.0
6       Modern European Restaurant   0.0
7                Mobile Phone Shop   0.0
8        Middle Eastern Restaurant   0.0
9                           Market   0.0


----Runnymede, Swansea----
                    

                             venue  freq
0                      Pizza Place  0.25
1                     Intersection  0.12
2                   Sandwich Place  0.12
3        Middle Eastern Restaurant  0.12
4                      Coffee Shop  0.12
5               Chinese Restaurant  0.12
6                   Discount Store  0.12
7  Molecular Gastronomy Restaurant  0.00
8                            Motel  0.00
9              Moroccan Restaurant  0.00


----Weston----
                             venue  freq
0                             Park   0.5
1                Convenience Store   0.5
2                Accessories Store   0.0
3        Middle Eastern Restaurant   0.0
4              Monument / Landmark   0.0
5  Molecular Gastronomy Restaurant   0.0
6       Modern European Restaurant   0.0
7                Mobile Phone Shop   0.0
8               Miscellaneous Shop   0.0
9                    Metro Station   0.0


----Wexford, Maryvale----
                       venue  freq
0  Middle Eastern 

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Gym,Pool,Coffee Shop,Skating Rink,Pharmacy,Pub,Sandwich Place,Dessert Shop,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Park,Supermarket,Shopping Mall,Bridal Shop,Sandwich Place,Restaurant,Pizza Place,Pharmacy
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Grocery Store,Sushi Restaurant,Greek Restaurant,Pharmacy,Pizza Place,Pub,Café


In [25]:
#clusters
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [26]:
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Gym,Pool,Coffee Shop,Skating Rink,Pharmacy,Pub,Sandwich Place,Dessert Shop,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Park,Supermarket,Shopping Mall,Bridal Shop,Sandwich Place,Restaurant,Pizza Place,Pharmacy
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Grocery Store,Sushi Restaurant,Greek Restaurant,Pharmacy,Pizza Place,Pub,Café


In [27]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [28]:
toronto_merged = toronto_merged_df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how= 'right')

toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Intersection,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Spa,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Accessories Store,Event Space,Boutique,Furniture / Home Store,Coffee Shop,Vietnamese Restaurant,Donut Shop,Discount Store,Distribution Center
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Yoga Studio,Sushi Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Café,Portuguese Restaurant,Chinese Restaurant
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,Dessert Shop
7,M3B,North York,Don Mills,43.745906,-79.352188,1,Gym,Restaurant,Coffee Shop,Japanese Restaurant,Beer Store,Italian Restaurant,Sporting Goods Shop,Bike Shop,Sandwich Place,Café
13,M3C,North York,Don Mills,43.725900,-79.340923,1,Gym,Restaurant,Coffee Shop,Japanese Restaurant,Beer Store,Italian Restaurant,Sporting Goods Shop,Bike Shop,Sandwich Place,Café
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Gym / Fitness Center,Café,Athletics & Sports,Gastropub,Bank,Intersection,Pharmacy,Pet Store,Breakfast Spot
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Lingerie Store,Italian Restaurant,Pizza Place


# Toronto Map

In [176]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=10)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

# Other Data

In [38]:
! wget https://en.wikipedia.org/wiki/List_of_postcode_districts_in_the_United_Kingdom

--2020-08-29 17:08:10--  https://en.wikipedia.org/wiki/List_of_postcode_districts_in_the_United_Kingdom
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘List_of_postcode_districts_in_the_United_Kingdom’

    [ <=>                                   ] 403,364     --.-K/s   in 0.1s    

2020-08-29 17:08:10 (2.91 MB/s) - ‘List_of_postcode_districts_in_the_United_Kingdom’ saved [403364]



In [51]:
!pip install beautifulsoup4
import requests # library to handle requests
from bs4 import BeautifulSoup #library for pulling data out of HTML and XML files.

url = "https://en.wikipedia.org/wiki/List_of_postcode_districts_in_the_United_Kingdom"
req = requests.get(url)
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[1]
df = pd.read_html(str(table))
eng_df = pd.DataFrame(df[0])



In [52]:
eng_df

,Postcode area,Postcode districts,Post town,Former postal county
0,AB,"AB10, AB11, AB12, AB15, AB16, AB21, AB22, AB23...",Aberdeen,(Aberdeenshire)
1,AB,AB13,Milltimber,Aberdeenshire
2,AB,AB14,Peterculter,Aberdeenshire
3,AB,AB30,Laurencekirk,Kincardineshire
4,AB,AB31,Banchory,Kincardineshire
5,AB,AB32,WESTHILL,Aberdeenshire
6,AB,AB33,ALFORD,Aberdeenshire
7,AB,AB34,Aboyne,(Aberdeenshire)
8,AB,AB35,Ballater,Aberdeenshire
9,AB,AB36,Strathdon,(Aberdeenshire)


In [53]:
eng_df.columns = ['Postal Code Area', 'Post Code District', 'Post Town', 'Formal Postal Country']
eng_df


,Postal Code Area,Post Code District,Post Town,Formal Postal Country
0,AB,"AB10, AB11, AB12, AB15, AB16, AB21, AB22, AB23...",Aberdeen,(Aberdeenshire)
1,AB,AB13,Milltimber,Aberdeenshire
2,AB,AB14,Peterculter,Aberdeenshire
3,AB,AB30,Laurencekirk,Kincardineshire
4,AB,AB31,Banchory,Kincardineshire
5,AB,AB32,WESTHILL,Aberdeenshire
6,AB,AB33,ALFORD,Aberdeenshire
7,AB,AB34,Aboyne,(Aberdeenshire)
8,AB,AB35,Ballater,Aberdeenshire
9,AB,AB36,Strathdon,(Aberdeenshire)


In [54]:
! wget https://www.freemaptools.com/download/outcode-postcodes/postcode-outcodes.csv

--2020-08-29 17:24:27--  https://www.freemaptools.com/download/outcode-postcodes/postcode-outcodes.csv
Resolving www.freemaptools.com (www.freemaptools.com)... 208.97.137.162, 2607:f298:5:101a::e06:9b6c
Connecting to www.freemaptools.com (www.freemaptools.com)|208.97.137.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84124 (82K) [text/csv]
Saving to: ‘postcode-outcodes.csv.1’

100%[======================================>] 84,124      --.-K/s   in 0.1s    

2020-08-29 17:24:27 (587 KB/s) - ‘postcode-outcodes.csv.1’ saved [84124/84124]



In [55]:
eng_geo = pd.read_csv('https://www.freemaptools.com/download/outcode-postcodes/postcode-outcodes.csv')
eng_geo

,id,postcode,latitude,longitude
0,2,AB10,57.135140,-2.117310
1,3,AB11,57.138750,-2.090890
2,4,AB12,57.101000,-2.110600
3,5,AB13,57.108010,-2.237760
4,6,AB14,57.100760,-2.270730
5,7,AB15,57.138680,-2.165250
6,8,AB16,57.161150,-2.155430
7,9,AB21,57.209600,-2.200330
8,10,AB22,57.187240,-2.119130
9,11,AB23,57.212420,-2.087760


In [61]:
eng_geo.columns = ['id', 'Post Code District', 'Latitude', 'Longitude']
eng_geo

,id,Post Code District,Latitude,Longitude
0,2,AB10,57.135140,-2.117310
1,3,AB11,57.138750,-2.090890
2,4,AB12,57.101000,-2.110600
3,5,AB13,57.108010,-2.237760
4,6,AB14,57.100760,-2.270730
5,7,AB15,57.138680,-2.165250
6,8,AB16,57.161150,-2.155430
7,9,AB21,57.209600,-2.200330
8,10,AB22,57.187240,-2.119130
9,11,AB23,57.212420,-2.087760


In [62]:
eng_merged_data = pd.merge(eng_df, eng_geo, on= 'Post Code District')
eng_merged_data

,Postal Code Area,Post Code District,Post Town,Formal Postal Country,id,Latitude,Longitude
0,AB,AB13,Milltimber,Aberdeenshire,5,57.10801,-2.23776
1,AB,AB14,Peterculter,Aberdeenshire,6,57.10076,-2.27073
2,AB,AB30,Laurencekirk,Kincardineshire,14,56.84678,-2.47712
3,AB,AB31,Banchory,Kincardineshire,15,57.07479,-2.52623
4,AB,AB32,WESTHILL,Aberdeenshire,16,57.15545,-2.31742
5,AB,AB33,ALFORD,Aberdeenshire,17,57.22464,-2.74203
6,AB,AB34,Aboyne,(Aberdeenshire),18,57.09393,-2.81204
7,AB,AB35,Ballater,Aberdeenshire,19,57.03829,-3.14869
8,AB,AB36,Strathdon,(Aberdeenshire),20,57.19715,-3.06589
9,AB,AB37,Ballindalloch,Banffshire,21,57.33098,-3.35051


In [63]:
CLIENT_ID = 'LPJNPAKLUKGOPLMLQK5VN4IIXDGZG4TD1G4CIRUZ2041M32T' # your Foursquare ID
CLIENT_SECRET = 'NDO54RFAMEAFLB3J2QRSB0B5K5DM3OI5DRXATAYQCWGG3XQN' # your Foursquare Secret
VERSION = '20200815' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LPJNPAKLUKGOPLMLQK5VN4IIXDGZG4TD1G4CIRUZ2041M32T
CLIENT_SECRET:NDO54RFAMEAFLB3J2QRSB0B5K5DM3OI5DRXATAYQCWGG3XQN


In [77]:
eng_merged_data.loc[0, 'Post Town']

'Milltimber'

In [79]:
eng_town_lat = eng_merged_data.loc[0, 'Latitude'] # town latitude value
eng_town_long = eng_merged_data.loc[0, 'Longitude'] # town longitude value

eng_town_name = eng_merged_data.loc[0, 'Post Town'] # town name

print('Latitude and longitude values of {} are {}, {}.'.format(eng_town_name, 
                                                               eng_town_lat, 
                                                               eng_town_long))

Latitude and longitude values of Milltimber are 57.10801, -2.2377599999999997.


In [99]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 2000 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    eng_town_lat, 
    eng_town_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LPJNPAKLUKGOPLMLQK5VN4IIXDGZG4TD1G4CIRUZ2041M32T&client_secret=NDO54RFAMEAFLB3J2QRSB0B5K5DM3OI5DRXATAYQCWGG3XQN&v=20200815&ll=57.10801,-2.2377599999999997&radius=2000&limit=100'

In [100]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5f4aa0b8219a894aef973992'},
 'response': {'headerLocation': 'Aberdeen',
  'headerFullLocation': 'Aberdeen',
  'headerLocationGranularity': 'city',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 57.12601001800002,
    'lng': -2.2046761657900795},
   'sw': {'lat': 57.090009981999984, 'lng': -2.27084383420992}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd4827f637ba59314cef470',
       'name': 'The Ploughman',
       'location': {'address': 'North Deeside Road',
        'lat': 57.09767806770377,
        'lng': -2.256617546081543,
        'labeledLatLngs': [{'label': 'display',
          'lat': 57.09767806770377,
          'lng': -2.256617546081543}],
        'distance': 1619,
        'cc': 'GB',
        'city': 'Petercul

In [101]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [102]:
venues = results['response']['groups'][0]['items']
    
eng_nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
eng_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
eng_nearby_venues =eng_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
eng_nearby_venues['venue.categories'] = eng_nearby_venues.apply(get_category_type, axis=1)

# clean columns
eng_nearby_venues.columns = [col.split(".")[-1] for col in eng_nearby_venues.columns]
eng_nearby_venues

,name,categories,lat,lng
0,The Ploughman,Pub,57.097678,-2.256618
1,Kippie Lodge,Gym / Fitness Center,57.102848,-2.250152
2,Terroir Cafe / Deli,Café,57.099225,-2.254815
3,Old Murtle Station,Trail,57.107579,-2.211490
4,River Dee,River,57.097862,-2.258018
5,Culter Curry House,Indian Restaurant,57.099988,-2.265646


In [103]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


3 venues were returned by Foursquare.


In [107]:
eng_venues = getNearbyVenues(names=eng_merged_data['Post Town'],
                                   latitudes=eng_merged_data['Latitude'],
                                   longitudes=eng_merged_data['Longitude']
                                  )


Milltimber
Peterculter
Laurencekirk
Banchory
WESTHILL
ALFORD
Aboyne
Ballater
Strathdon
Ballindalloch
Aberlour
Stonehaven
ELLON
Peterhead
Fraserburgh
MACDUFF
BANFF
Inverurie
Insch
Turriff
Huntly
KEITH
Buckie
HATFIELD
Alcester
Frome
Warminster
WESTBURY
Trowbridge
BRADFORD-ON-AVON
STREET
Cleckheaton
BROADSTONE
Swanage
WAREHAM
WIMBORNE
Ferndown
CHRISTCHURCH
Ringwood
New Milton
Verwood
Peacehaven
Arundel
Pevensey
SEAFORD
Polegate
Hailsham
Shoreham-by-Sea
Steyning
Clevedon
Winscombe
Axbridge
CHEDDAR
Wedmore
Banwell
Holywood
Donaghadee
BALLYNAHINCH
DROMORE
HILLSBOROUGH
Downpatrick
Castlewellan
Banbridge
NEWCASTLE
Newry
Newry
Carrickfergus
Ballyclare
Larne
ANTRIM
Magherafelt
Maghera
Limavady
Ballymoney
BALLYCASTLE
Portstewart
Portrush
Bushmills
CALEDON
AUGHNACLOY
Fivemiletown
Clogher
Augher
Cookstown
Castlederg
Strabane
KESWICK
Cockermouth
Maryport
Appleby-in-Westmorland
Kirkby Stephen
Ravenglass
Holmrook
Seascale
Beckermet
EGREMONT
Cleator
Moor Row
Cleator Moor
Frizington
ST. BEES
Whitehaven


KeyError: 'groups'

In [108]:
print(eng_venues.shape)
eng_venues

(2046, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Peterculter,57.10076,-2.27073,Culter Curry House,57.099988,-2.265646,Indian Restaurant
1,MACDUFF,57.66877,-2.49122,Macduff Aquarium,57.672095,-2.492021,Aquarium
2,MACDUFF,57.66877,-2.49122,The Knowes Hotel Banff and Macduff,57.669562,-2.495662,Hotel
3,MACDUFF,57.66877,-2.49122,Munro Landscapes,57.671236,-2.496629,Construction & Landscaping
4,MACDUFF,57.66877,-2.49122,Echomaster Marine Ltd,57.670220,-2.497901,Electronics Store
5,Inverurie,57.28907,-2.40485,J G Ross,57.284916,-2.404139,Café
6,Inverurie,57.28907,-2.40485,Busy Bees at Inverurie,57.286519,-2.398112,Nursery School
7,Insch,57.34396,-2.60651,Cafe mini indian take away and restaurant,57.345885,-2.605647,Indian Restaurant
8,HATFIELD,51.75958,-0.22920,Hatfield Swim Centre,51.762070,-0.231266,Pool
9,HATFIELD,51.75958,-0.22920,聚点 Here Japanese Restaurant,51.762788,-0.225269,Japanese Restaurant


In [133]:
eng_venues.groupby('Neighbourhood').count()
print('There are {} uniques categories.'.format(len(eng_venues['Venue Category'].unique())))

eng_onehot = pd.get_dummies(eng_venues[['Venue Category']], prefix="", prefix_sep="")

eng_onehot['Post Town'] = eng_venues['Neighbourhood'] 

eng_fixed_columns = [eng_onehot.columns[-1]] + list(eng_onehot.columns[:-1])
eng_onehot = eng_onehot[eng_fixed_columns]

eng_onehot.head()

There are 231 uniques categories.


,Post Town,Airport,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Video Game Store,Warehouse Store,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Peterculter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MACDUFF,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MACDUFF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MACDUFF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MACDUFF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
eng_grouped = eng_onehot.groupby('Post Town').mean().reset_index()
eng_grouped

,Post Town,Airport,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Video Game Store,Warehouse Store,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ABERDYFI,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,ALFORD,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.0,1.0,0.0,0.000000,0.0,0.0,0.0
2,ALTON,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,ALVA,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,ANNAN,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5,ASHFORD,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
6,Aberdare,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
7,Abertillery,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
8,Addlestone,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
9,Alcester,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [136]:

num_top_venues = 4

for town in eng_grouped['Post Town']:
    print("----"+town+"----")
    eng_temp = eng_grouped[eng_grouped['Post Town'] == town].T.reset_index()
    eng_temp.columns = ['venue','freq']
    eng_temp = eng_temp.iloc[1:]
    eng_temp['freq'] = eng_temp['freq'].astype(float)
    eng_temp = eng_temp.round({'freq': 2})
    print(eng_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ABERDYFI----
            venue  freq
0           Hotel   0.2
1             Pub   0.2
2  Ice Cream Shop   0.2
3   Train Station   0.2


----ALFORD----
            venue  freq
0        Windmill   1.0
1         Airport   0.0
2        Pharmacy   0.0
3  Nursery School   0.0


----ALTON----
            venue  freq
0        Pharmacy   0.5
1       Gastropub   0.5
2       Pet Store   0.0
3  Nursery School   0.0


----ALVA----
               venue  freq
0  Indian Restaurant  0.33
1      Grocery Store  0.33
2              Trail  0.33
3            Airport  0.00


----ANNAN----
         venue  freq
0          Bar  0.33
1    Hotel Bar  0.33
2  Pizza Place  0.33
3      Airport  0.00


----ASHFORD----
         venue  freq
0          Pub   0.6
1  Coffee Shop   0.2
2         Park   0.2
3      Airport   0.0


----Aberdare----
         venue  freq
0         Park  0.25
1          Pub  0.25
2  Supermarket  0.25
3      Theater  0.25


----Abertillery----
                        venue  freq
0             

           venue  freq
0  Grocery Store  0.33
1           Park  0.33
2       Gym Pool  0.33
3        Airport  0.00


----Belper----
               venue  freq
0                Pub  0.33
1      Movie Theater  0.17
2  Indian Restaurant  0.17
3        Pizza Place  0.17


----Bembridge----
               venue  freq
0  Convenience Store  0.25
1          Gastropub  0.25
2  Fish & Chips Shop  0.25
3               Park  0.25


----Beverley----
         venue  freq
0          Pub  0.33
1  Supermarket  0.33
2          Bar  0.17
3         Café  0.17


----Bideford----
        venue  freq
0         Pub  0.33
1       Hotel  0.33
2  Theme Park  0.33
3     Airport  0.00


----Biggleswade----
               venue  freq
0        Flea Market  0.14
1           Platform  0.14
2  Indian Restaurant  0.14
3        Supermarket  0.14


----Billingshurst----
            venue  freq
0             Pub   1.0
1         Airport   0.0
2       Pet Store   0.0
3  Nursery School   0.0


----Bilston----
               v

                venue  freq
0         Coffee Shop   0.4
1      Sandwich Place   0.2
2           Multiplex   0.2
3  Chinese Restaurant   0.2


----Corbridge----
                venue  freq
0                 Pub   0.4
1  Italian Restaurant   0.1
2              Bakery   0.1
3                Café   0.1


----Corrour----
            venue  freq
0            Lake   1.0
1         Airport   0.0
2        Pharmacy   0.0
3  Nursery School   0.0


----Cowes----
               venue  freq
0        Supermarket   0.4
1  Convenience Store   0.2
2                Gym   0.2
3    Automotive Shop   0.2


----Cramlington----
               venue  freq
0                Bar  0.25
1  Convenience Store  0.25
2               Park  0.25
3         Skate Park  0.25


----Crewkerne----
                        venue  freq
0  Construction & Landscaping   0.5
1                 Pizza Place   0.5
2                     Airport   0.0
3                   Pet Store   0.0


----Criccieth----
                        venue  fre

         venue  freq
0     Pharmacy  0.17
1  Supermarket  0.17
2   Restaurant  0.17
3     Bus Stop  0.17


----Forest Row----
                        venue  freq
0                        Café  0.22
1          Chinese Restaurant  0.11
2  Construction & Landscaping  0.11
3                 Pizza Place  0.11


----Forres----
            venue  freq
0      Restaurant  0.25
1           Hotel  0.25
2  Scenic Lookout  0.25
3            Park  0.25


----Fort Augustus----
    venue  freq
0     Pub  0.19
1  Resort  0.06
2   Hotel  0.06
3    Café  0.06


----Fortrose----
            venue  freq
0   Grocery Store   1.0
1         Airport   0.0
2       Pet Store   0.0
3  Nursery School   0.0


----Fowey----
                venue  freq
0                 Pub  0.23
1               Hotel  0.23
2  Seafood Restaurant  0.15
3       Boat or Ferry  0.15


----Frinton-on-Sea----
                     venue  freq
0            Jewelry Store  0.12
1          Nature Preserve  0.12
2  Fruit & Vegetable Store  0.12
3

            venue  freq
0             Bar   1.0
1         Airport   0.0
2       Pet Store   0.0
3  Nursery School   0.0


----Johnstone----
         venue  freq
0          Pub  0.25
1  Music Venue  0.25
2         Café  0.25
3         Park  0.25


----KESWICK----
         venue  freq
0          Pub  0.22
1        Hotel  0.10
2  Coffee Shop  0.08
3         Café  0.08


----KIRKNEWTON----
               venue  freq
0  Fish & Chips Shop   1.0
1            Airport   0.0
2          Pet Store   0.0
3     Nursery School   0.0


----Kilbirnie----
            venue  freq
0        Gym Pool  0.25
1     Supermarket  0.25
2    Home Service  0.25
3  Clothing Store  0.25


----Kilgetty----
            venue  freq
0       Gastropub   1.0
1         Airport   0.0
2       Pet Store   0.0
3  Nursery School   0.0


----Killin----
                     venue  freq
0                    Hotel  0.29
1  Comfort Food Restaurant  0.14
2            Grocery Store  0.14
3           Scenic Lookout  0.14


----Kilmacolm

                  venue  freq
0     Convenience Store   0.5
1  Gym / Fitness Center   0.5
2               Airport   0.0
3              Pharmacy   0.0


----Milford Haven----
              venue  freq
0  Business Service   1.0
1           Airport   0.0
2         Pet Store   0.0
3    Nursery School   0.0


----Mirfield----
       venue  freq
0  Gastropub   0.5
1        Bar   0.5
2    Airport   0.0
3  Nightclub   0.0


----Monmouth----
            venue  freq
0           Hotel   1.0
1         Airport   0.0
2        Pharmacy   0.0
3  Nursery School   0.0


----Montacute----
           venue  freq
0          Hotel   0.5
1  Historic Site   0.5
2        Airport   0.0
3       Pharmacy   0.0


----Moreton-in-Marsh----
         venue  freq
0        Hotel  0.23
1          Pub  0.15
2  Coffee Shop  0.15
3         Café  0.08


----Much Hadham----
               venue  freq
0                Pub   0.2
1  Convenience Store   0.2
2               Food   0.2
3   Business Service   0.2


----Musselburgh--

                        venue  freq
0  Construction & Landscaping  0.50
1           Convenience Store  0.25
2                        Park  0.25
3                     Airport  0.00


----ROSLIN----
            venue  freq
0   Bowling Green   1.0
1       Pet Store   0.0
2    Noodle House   0.0
3  Nursery School   0.0


----RYTON----
         venue  freq
0  Rugby Pitch   0.5
1  Pizza Place   0.5
2      Airport   0.0
3    Pet Store   0.0


----Ravenglass----
                        venue  freq
0                      Castle   0.5
1  Construction & Landscaping   0.5
2                     Airport   0.0
3        Other Great Outdoors   0.0


----Retford----
            venue  freq
0  Soccer Stadium   1.0
1         Airport   0.0
2       Pet Store   0.0
3  Nursery School   0.0


----Ringwood----
               venue  freq
0  Convenience Store  0.25
1            Brewery  0.12
2               Café  0.12
3  Fish & Chips Shop  0.12


----Rowlands Gill----
            venue  freq
0     Pizza Place   1

                  venue  freq
0  Gym / Fitness Center  0.50
1                   Pub  0.25
2                  Park  0.25
3               Airport  0.00


----Swaffham----
         venue  freq
0  Rugby Pitch   0.5
1         Café   0.5
2      Airport   0.0
3     Pharmacy   0.0


----Swanage----
           venue  freq
0            Pub   0.2
1    Golf Course   0.2
2  Grocery Store   0.2
3     Skate Park   0.2


----TALYBONT----
                venue  freq
0  Italian Restaurant  0.17
1      Ice Cream Shop  0.17
2          Campground  0.17
3       Train Station  0.17


----TARBERT----
            venue  freq
0   Boat or Ferry   1.0
1         Airport   0.0
2       Pet Store   0.0
3  Nursery School   0.0


----TAVISTOCK----
            venue  freq
0    Home Service   1.0
1         Airport   0.0
2        Pharmacy   0.0
3  Nursery School   0.0


----TIVERTON----
              venue  freq
0  Toy / Game Store   0.5
1             Hotel   0.5
2         Pet Store   0.0
3      Noodle House   0.0


----T

               venue  freq
0  Convenience Store   0.5
1              Beach   0.5
2            Airport   0.0
3           Pharmacy   0.0


----West Byfleet----
             venue  freq
0  Harbor / Marina   0.2
1            River   0.2
2             Park   0.2
3   Cricket Ground   0.2


----West Calder----
                 venue  freq
0    Indian Restaurant  0.33
1          Men's Store  0.17
2  Arts & Crafts Store  0.17
3        Grocery Store  0.17


----West Kilbride----
           venue  freq
0          Beach  0.50
1   Home Service  0.25
2  Grocery Store  0.25
3        Airport  0.00


----Westerham----
               venue  freq
0  Indian Restaurant   1.0
1            Airport   0.0
2           Pharmacy   0.0
3     Nursery School   0.0


----Weybridge----
               venue  freq
0      Grocery Store  0.17
1  Indian Restaurant  0.17
2     Cricket Ground  0.08
3     Sandwich Place  0.08


----Whitehaven----
                  venue  freq
0                 Hotel  0.25
1         Train Stat

In [158]:
def return_most_common_venues(row, num_top_venues):
    eng_row_categories = row.iloc[1:]
    eng_row_categories_sorted = eng_row_categories.sort_values(ascending=False)
    
    return eng_row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
eng_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
eng_neighborhoods_venues_sorted['Neighbourhood'] = eng_grouped['Post Town']

for ind in np.arange(eng_grouped.shape[0]):
    eng_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eng_grouped.iloc[ind, :], num_top_venues)

eng_neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABERDYFI,Ice Cream Shop,Hotel,Café,Train Station,Pub,Hostel,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop
1,ALFORD,Windmill,Zoo,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service
2,ALTON,Gastropub,Pharmacy,Zoo,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service
3,ALVA,Indian Restaurant,Grocery Store,Trail,Entertainment Service,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
4,ANNAN,Hotel Bar,Pizza Place,Bar,Zoo,Entertainment Service,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market


In [159]:
#clusters
from sklearn.cluster import KMeans
eng_kclusters = 5

eng_grouped_clustering = eng_grouped.drop('Post Town', 1)

eng_kmeans = KMeans(n_clusters=eng_kclusters, random_state=0).fit(eng_grouped_clustering)

eng_kmeans.labels_[0:10]

array([1, 0, 0, 0, 0, 2, 0, 2, 0, 4], dtype=int32)

In [160]:
eng_neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABERDYFI,Ice Cream Shop,Hotel,Café,Train Station,Pub,Hostel,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop
1,ALFORD,Windmill,Zoo,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service
2,ALTON,Gastropub,Pharmacy,Zoo,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service
3,ALVA,Indian Restaurant,Grocery Store,Trail,Entertainment Service,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
4,ANNAN,Hotel Bar,Pizza Place,Bar,Zoo,Entertainment Service,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market


In [161]:
eng_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', eng_kmeans.labels_)


In [163]:
eng_merged = eng_merged_data
eng_merged = eng_merged.join(eng_neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Post Town', how= 'right')

eng_merged

,Postal Code Area,Post Code District,Post Town,Formal Postal Country,id,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,AB,AB14,Peterculter,Aberdeenshire,6,57.10076,-2.27073,0,Indian Restaurant,French Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service,Film Studio
5,AB,AB33,ALFORD,Aberdeenshire,17,57.22464,-2.74203,0,Windmill,Zoo,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service
390,LN,LN13,ALFORD,Lincolnshire,1479,53.26518,0.18482,0,Windmill,Zoo,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service
15,AB,AB44,MACDUFF,Banffshire,27,57.66877,-2.49122,1,Electronics Store,Construction & Landscaping,Aquarium,Hotel,Fish & Chips Shop,Event Service,Farm,Fast Food Restaurant,Film Studio,Financial or Legal Service
17,AB,AB51,Inverurie,Aberdeenshire,29,57.28907,-2.40485,0,Nursery School,Café,Zoo,Event Service,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
18,AB,AB52,Insch,Aberdeenshire,30,57.34396,-2.60651,0,Indian Restaurant,French Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service,Film Studio
23,AL,AL10,HATFIELD,Hertfordshire,36,51.75958,-0.22920,0,Grocery Store,Discount Store,Middle Eastern Restaurant,Pool,Japanese Restaurant,Noodle House,Sandwich Place,Supermarket,Pharmacy,Film Studio
24,B,B50,Alcester,Warwickshire,87,52.16847,-1.85377,4,Construction & Landscaping,Zoo,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service
25,BA,BA11,Frome,Somerset,124,51.23652,-2.32519,2,Pub,Zoo,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service
27,BA,BA13,WESTBURY,Wiltshire,126,51.26201,-2.17917,0,Supermarket,Pub,Breakfast Spot,Grocery Store,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service


In [177]:
# create map
eng_map_clusters = folium.Map(location=[51.509865, -0.118092], zoom_start=10)


x = np.arange(eng_kclusters)
ys = [i + x + (i*x)**2 for i in range(eng_kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
eng_markers_colors = []
for lat, lon, poi, cluster in zip(eng_merged['Latitude'], eng_merged['Longitude'], eng_merged['Post Town'], eng_merged['Cluster Labels']):
    eng_label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=eng_label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(eng_map_clusters)
eng_map_clusters